In [ ]:
# ===================================================================
# CELL 1: SETUP AND CLONE
# ===================================================================
import os
import sys
import base64
from kaggle_secrets import UserSecretsClient

# --- 1. Retrieve ALL Secrets ---
print("--- 1. Retrieving secrets from Kaggle Secret Manager ---")
user_secrets = UserSecretsClient()
git_pat = user_secrets.get_secret("GITHUB_PAT")
git_key_b64 = user_secrets.get_secret("GIT_DEPLOY_KEY_B64")
gdrive_creds_b64 = user_secrets.get_secret("GDRIVE_CREDENTIALS_DATA_B64")

# --- 2. Clone the Private Repository using HTTPS and Personal Access Token ---
print("\n--- 2. Cloning the private project repository ---")
# --- IMPORTANT: Replace with YOUR GitHub username and repo name ---
GIT_USERNAME = "labyedh"
GIT_REPO = "Alzheimer_classification_mlops"
PROJECT_DIR = "Alzheimer_classification_mlops"

# This is the HTTPS URL format for cloning with a PAT
GIT_URL_HTTPS = f"https://{git_pat}@github.com/{GIT_USERNAME}/{GIT_REPO}.git"

# Clean up and clone
!rm -rf {PROJECT_DIR}
!git clone {GIT_URL_HTTPS}
%cd {PROJECT_DIR}

# --- 3. Set up SSH for PUSHING back to GitHub ---
print("\n--- 3. Configuring SSH for secure push ---")
ssh_dir = "/root/.ssh"
os.makedirs(ssh_dir, exist_ok=True)
key_path = os.path.join(ssh_dir, "id_ed25519")
with open(key_path, "w") as f:
    f.write(base64.b64decode(git_key_b64).decode())
os.chmod(key_path, 0o600)
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# --- 4. Switch Git remote from HTTPS to SSH for pushing ---
# This uses the more secure, repo-specific Deploy Key for write access
GIT_URL_SSH = f"git@github.com:{GIT_USERNAME}/{GIT_REPO}.git"
!git remote set-url origin {GIT_URL_SSH}

# --- 5. Write DVC Credentials into the project folder ---
print("\n--- 5. Writing DVC Credentials ---")
creds_path = "gdrive-credentials.json" # We are already inside the project dir
with open(creds_path, "w") as f:
    f.write(base64.b64decode(gdrive_creds_b64).decode())

print("\n--- Setup Cell Finished ---")

In [ ]:
# ===================================================================
# CELL 3: RUN PIPELINE AND PUSH RESULTS
# ===================================================================
import os
import sys

# Change into the project directory (if this is a separate cell)


# ... (Installation and DVC commands) ...
!pip install -r requirements.txt --quiet

print("\n--- Pulling DVC data ---")
!dvc pull --force

print("\n--- Reproducing DVC pipeline ---")
!dvc repro

print("\n--- Pushing DVC artifacts ---")
# Step 1: Pushes large files (models, plots) to Google Drive
!dvc push

print("\n--- Committing results back to Git ---")
# Step 2: Configure Git identity for the commit
!git config --global user.name "yessine_labyedh"
!git config --global user.email "labyedh@fsb.u-carthage.tn"

# Step 3: Add the files that DVC and your scripts have changed
!git add dvc.lock metrics/ plots/ .gitignore

# Step 4: Commit the changes
# The `|| true` part prevents an error if there are no new changes to commit
!git commit -m "CI: Automated run from Kaggle GPU" || true

# Step 5: PUSH TO GITHUB REPOSITORY (THE FINAL STEP)
print("\n--- Pushing changes to GitHub ---")
!git push

print("\n\n--- Workflow Finished Successfully ---")